<a href="https://colab.research.google.com/github/thomreid11/DAS7000_PORT1/blob/main/DAS7000_port1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thomas Reid - DAS7000 - PORT1**

Exploratory Data Analysis (EDA) on Manhattan Property Dataset investigating property types and prices within Manhattan, New York

Link to data set: https://drive.google.com/file/d/1XRVnZ2Z_XkD1dDmF1uYBdjjF2WVAzg0g/view?usp=drive_link

#### **Importing necessary libraries**

In [37]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

#### **Mounting Google Drive to Google Collab and importing dataset**

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
drive_path = '/content/drive/MyDrive/Manhattan property dataset csv.csv'

In [40]:
# Converting csv to df
df = pd.read_csv(drive_path)

In [41]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASEMENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,376,43,NaN,S1,743 EAST 6 STREET,NaN,...,1.0,1.0,2.0,"2,090","3,680",1940.0,1,S1,0,01/23/2025
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,372,19,NaN,C7,"292 EAST 3 STREET, 4B",NaN,...,9.0,1.0,10.0,"2,401","6,920",1920.0,2,C7,0,02/07/2025
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,396,26,NaN,C4,638 EAST 14 STREET,NaN,...,20.0,0.0,20.0,"2,779","10,740",1920.0,2,C4,0,05/01/2025
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,396,27,NaN,C4,640 EAST 14 STREET,NaN,...,20.0,0.0,20.0,"2,831","10,900",1920.0,2,C4,0,05/01/2025
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,400,11,NaN,C4,"510 EAST 5TH STREET, 9",NaN,...,18.0,0.0,18.0,"2,404","6,875",1900.0,2,C4,"280,000",04/29/2025


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18491 entries, 0 to 18490
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         18491 non-null  int64  
 1   NEIGHBORHOOD                    18491 non-null  object 
 2   BUILDING CLASS CATEGORY         18491 non-null  object 
 3   TAX CLASS AT PRESENT            18491 non-null  object 
 4   BLOCK                           18491 non-null  int64  
 5   LOT                             18491 non-null  int64  
 6   EASEMENT                        0 non-null      float64
 7   BUILDING CLASS AT PRESENT       18491 non-null  object 
 8   ADDRESS                         18491 non-null  object 
 9   APARTMENT NUMBER                8634 non-null   object 
 10  ZIP CODE                        18491 non-null  int64  
 11  RESIDENTIAL UNITS               9366 non-null   float64
 12  COMMERCIAL UNITS                

#### **Data Preprocessing**

In [43]:
# Making columns neat and uniform
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.title()

In [44]:
# Correcting the data types
# Changing strings to floats and removing the commas
df['Land_Square_Feet'] = df['Land_Square_Feet'].str.replace(',', '', regex=False).astype(float)
df['Gross_Square_Feet'] = df['Gross_Square_Feet'].str.replace(',', '', regex=False).astype(float)
df['Sale_Price'] = df['Sale_Price'].str.replace(',', '', regex=False).astype(int)

# Changing the remaining data types
df['Sale_Date'] = pd.to_datetime(df['Sale_Date'])
df["Tax_Class_At_Time_Of_Sale"] = df["Tax_Class_At_Time_Of_Sale"].astype("string")
df["Tax_Class_At_Present"] = df["Tax_Class_At_Present"].astype("string")


# Changing 3 columns in one go for efficiency
unit_cols = ["Residential_Units", "Commercial_Units", "Total_Units"]
for col in unit_cols:
    df[col] = df[col].astype("Int64")


In [45]:
# Number of Rows
df.shape[0]

18491

In [46]:
# Number of Columns
df.shape[1]

21

We can see that there are 20 columns and 18491. The list shows that not all columns have the full amount of data. The amount of entries missing per column will be worked out and then if it is over 50%, the column will be dropped. If not then it will be interpolated.

In [47]:
# Finding out the percentage of missing values per column
def missing_values_percentage_table(df):

# Finding the number of missing values per column
    missing_values = df.isnull().sum()

# Finding the percentage of missing values by dividing it by the number of data entries there should be and multiplying by 100
    missing_value_perc =  (100 * missing_values / len(df))

# Concatenating the missing values and missing values %  along the top of the table
    missing_value_table = pd.concat([missing_values, missing_value_perc], axis=1)
    missing_value_table = missing_value_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})

# Sort the table by percentage of missing descending
    missing_values_table = missing_value_table.sort_values('% of Total Values', ascending=False)
    return missing_values_table

missing_percentage_df = missing_values_percentage_table(df)


display(missing_percentage_df.style.background_gradient(cmap='Blues'))

,Missing Values,% of Total Values
Easement,18491,100.000000
Land_Square_Feet,16963,91.736520
Gross_Square_Feet,16963,91.736520
Commercial_Units,16171,87.453356
Apartment_Number,9857,53.307014
Residential_Units,9125,49.348332
Total_Units,8333,45.065167
Year_Built,1936,10.469958
Block,0,0.000000
Borough,0,0.000000


In [48]:
# Omitting Easement, Land_Square_Feet, Gross_Square_Feet, Commerical_Units, Apartment_number, Residential_Units

cols_to_drop = [
    "Easement",
    "Land_Square_Feet",
    "Gross_Square_Feet",
    "Commercial_Units",
    "Apartment_Number",
    "Residential_Units"
]

df = df.drop(columns=cols_to_drop)

Total Units - Interpolated through using the median Total Units of the neighbourhood

Year Built - Interpolated through using the modal Year Built of building category

In [49]:
# Interpolating Total Units
df["Total_Units"] = df.groupby("Neighborhood")["Total_Units"]\
                      .transform(lambda x: x.fillna(x.median()))

In [50]:
# Interpolating Year Built
df["Year_Built"] = df.groupby("Building_Class_Category")["Year_Built"]\
                     .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.median()))


In [51]:
# Re-checking missing values
missing_values = df.isnull().sum()
missing_values

,0
Borough,0
Neighborhood,0
Building_Class_Category,0
Tax_Class_At_Present,0
Block,0
Lot,0
Building_Class_At_Present,0
Address,0
Zip_Code,0
Total_Units,0


In [52]:
# Within sale price column, some values show '0', this code removes their row from the dataframe
zeros = (df["Sale_Price"] == 0).sum()
print("Rows removed:", zeros)
df = df[df["Sale_Price"] != 0]


Rows removed: 4056


In [53]:
neighborhood_organising = {
    # Greenwich Village
    "GREENWICH VILLAGE-CENTRAL": "GREENWICH VILLAGE",
    "GREENWICH VILLAGE-WEST": "GREENWICH VILLAGE",

    # Harlem
    "HARLEM-CENTRAL": "HARLEM",
    "HARLEM-EAST": "HARLEM",
    "HARLEM-UPPER": "HARLEM",
    "HARLEM-WEST": "HARLEM",

    # Midtown
    "MIDTOWN EAST": "MIDTOWN",
    "MIDTOWN WEST": "MIDTOWN",

    # Upper East Side
    "UPPER EAST SIDE (59-79)": "EAST SIDE",
    "UPPER EAST SIDE (79-96)": "EAST SIDE",
    "UPPER EAST SIDE (96-110)": "EAST SIDE",

    # Upper West Side
    "UPPER WEST SIDE (59-79)": "WEST SIDE",
    "UPPER WEST SIDE (79-96)": "WEST SIDE",
    "UPPER WEST SIDE (96-116)": "WEST SIDE",

    # Washington Heights
    "WASHINGTON HEIGHTS LOWER": "WASHINGTON HEIGHTS",
    "WASHINGTON HEIGHTS UPPER": "WASHINGTON HEIGHTS",

    #East Side
    "LOWER EAST SIDE": "EAST SIDE",
    "UPPER EAST SIDE": "EAST SIDE",
    #West Side
    "LOWER WEST SIDE": "WEST SIDE",
    "UPPER WEST SIDE": "WEST SIDE"
}

df["Neighborhood"] = df["Neighborhood"].replace(neighborhood_organising)


In [54]:
df["Neighborhood"].unique()



array(['ALPHABET CITY', 'CHELSEA', 'CHINATOWN', 'CIVIC CENTER', 'CLINTON',
       'EAST VILLAGE', 'FASHION', 'FINANCIAL', 'FLATIRON', 'GRAMERCY',
       'GREENWICH VILLAGE', 'HARLEM', 'INWOOD', 'JAVITS CENTER',
       'KIPS BAY', 'LITTLE ITALY', 'EAST SIDE', 'MANHATTAN VALLEY',
       'MIDTOWN CBD', 'MIDTOWN', 'MORNINGSIDE HEIGHTS', 'MURRAY HILL',
       'ROOSEVELT ISLAND', 'SOHO', 'SOUTHBRIDGE', 'TRIBECA', 'WEST SIDE',
       'WASHINGTON HEIGHTS'], dtype=object)

In [55]:
# Making new csv with updated data entries
df_to_csv = df.to_csv('Manhattan_Property_Dataset_Updated.csv')

# **Data Analysis**

#### **Mean Sale Prices per Neighbourhood**

In [56]:
# Which neibourhood is the most expensive?

# Grouping neighborhoods and calculating mean sale price
neighborhood_mean = (
    df.groupby("Neighborhood", as_index=False)["Sale_Price"]
      .mean()
      .rename(columns={"Sale_Price": "Mean_Sale_Price"})
)

# Sort by mean sale price in descending order
neighborhood_mean = neighborhood_mean.sort_values(
    "Mean_Sale_Price", ascending=False
)

# Plot with Plotly Express
fig = px.bar(
    data_frame=neighborhood_mean, # Added data_frame argument
    x="Neighborhood",
    y="Mean_Sale_Price",
    title="Most Expensive Neighborhoods (Mean Sale Price)",
    labels={"Neighborhood": "Neighborhood", "Mean_Sale_Price": "Mean Sale Price ($)"},
)

fig.show()

#### **Number of Sales per Neighbourhood**

In [57]:
sales_per_neighborhood = (
    df.groupby("Neighborhood")
      .size()
      .reset_index(name="Number_of_Sales")
      .sort_values("Number_of_Sales", ascending=False)
)
fig = px.bar(
    sales_per_neighborhood,
    x="Number_of_Sales",
    y="Neighborhood",
    orientation="h",
    title="Number of Property Sales per Neighborhood"
)

fig.update_layout(
    height=1000,
    bargap=0.25,
    yaxis={'categoryorder':'total ascending'})
fig.show()

#### **Building Class Frequency**

Building Class Types Key:

* R - Condominiums (Condos)
* D - Elevator Apartments
* C - Walk-Up Apartments
* O - Office Buildings
* A - Single Family Homes
* B - Two Family Homes
* K - Mixed Use Buildings
* S - Cooperative Apartments




In [58]:
df["Building_Class_Group"] = df["Building_Class_At_Present"].str[0]
bc_group_counts = (
    df["Building_Class_Group"]
      .value_counts()
      .reset_index()
)
bc_group_counts.columns = ["Building_Class_Group", "Count"]
bc_group_counts = bc_group_counts[bc_group_counts['Count'] >= 50]

fig = px.bar(
    bc_group_counts,
    x="Count",
    y="Building_Class_Group",
    orientation="h",
    title="Frequency of Building Class Groups",
    labels={
        "Count": "Number of Sales",
        "Building_Class_Group": "Building Class Group"
    }
)

fig.update_layout(
    height=800,
    bargap=0.15,
    yaxis={"categoryorder": "total ascending"}
)

fig.show()

#### **Average Sale Price per Building Class**

In [59]:
class_counts = df["Building_Class_Group"].value_counts()
valid_classes = class_counts[class_counts > 50].index

df_filtered = df[df["Building_Class_Group"].isin(valid_classes)]
summary_table = (
    df_filtered
    .groupby("Building_Class_Group")["Sale_Price"]
    .agg(
        Mean="mean",
        Median="median",
        Lower_Quartile=lambda x: x.quantile(0.25),
        Upper_Quartile=lambda x: x.quantile(0.75),
        Count="count"
    )
    .sort_values("Mean", ascending=False)
)
summary_table = summary_table.round(2)
summary_table


,Mean,Median,Lower_Quartile,Upper_Quartile,Count
Building_Class_Group,,,,,
K,45482387.91,8250000.0,4275000.0,17750067.75,78
O,45203017.69,12000000.0,3537500.0,40375000.00,104
S,9099805.07,4800000.0,2325000.0,8500000.00,57
A,9084464.10,7650000.0,5030000.0,11625000.00,99
B,5174213.87,3995000.0,1906500.0,7050000.00,83
R,4672592.46,1575000.0,860000.0,3100000.00,7344
C,2824505.75,827500.0,420000.0,2550000.00,1036
D,2006579.23,879000.0,550000.0,1700000.00,5513


#### **Average Property Price per Neighbourhood over time**

In [64]:
df["Sale_Date"] = pd.to_datetime(df["Sale_Date"], errors="coerce")

# Extract year
df["Sale_Year"] = df["Sale_Date"].dt.year

yearly_neigh_mean = (
    df.groupby(["Sale_Year", "Neighborhood"], as_index=False)["Sale_Price"]
      .mean()
)

import plotly.graph_objects as go

fig = go.Figure()

for neigh in yearly_neigh_mean["Neighborhood"].unique():
    df_n = yearly_neigh_mean[yearly_neigh_mean["Neighborhood"] == neigh]
    fig.add_trace(
        go.Scatter(
            x=df_n["Sale_Year"],
            y=df_n["Sale_Price"],
            name=neigh,
            visible=True
        )
    )

buttons = [
    dict(
        label="All Neighborhoods",
        method="update",
        args=[{"visible": [True]*len(fig.data)}]
    ),
    dict(
        label="Hide All",
        method="update",
        args=[{"visible": [False]*len(fig.data)}]
    )
]

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        buttons=buttons,
        direction="down",
        x=1.15,
        y=1
    )],
    title="Mean Sale Price per Year by Neighborhood",
    yaxis_type="log"
)

fig.show()



In [67]:
import pandas as pd
import plotly.graph_objects as go


# Ensure datetime + numeric
df["Sale_Date"] = pd.to_datetime(df["Sale_Date"], errors="coerce")
df["Sale_Price"] = pd.to_numeric(df["Sale_Price"], errors="coerce")

# Extract year and remove invalid rows
df["Sale_Year"] = df["Sale_Date"].dt.year
df = df[(df["Sale_Year"].notna()) & (df["Sale_Price"] > 0)]

# ===== 2) Aggregate: mean sale price per year per neighborhood =====
yearly_neigh_mean = (
    df.groupby(["Sale_Year", "Neighborhood"], as_index=False)["Sale_Price"]
      .mean()
)

# Optional: keep only neighborhoods with enough data points (uncomment if needed)
# valid_neigh = yearly_neigh_mean["Neighborhood"].value_counts()
# yearly_neigh_mean = yearly_neigh_mean[yearly_neigh_mean["Neighborhood"].isin(valid_neigh[valid_neigh >= 3].index)]

# ===== 3) Build interactive dropdown (individual neighborhood selection) =====
fig = go.Figure()

neighborhoods = sorted(yearly_neigh_mean["Neighborhood"].dropna().unique())

# Add one trace per neighborhood (start hidden)
for neigh in neighborhoods:
    df_n = yearly_neigh_mean[yearly_neigh_mean["Neighborhood"] == neigh]
    fig.add_trace(
        go.Scatter(
            x=df_n["Sale_Year"],
            y=df_n["Sale_Price"],
            mode="lines",
            name=neigh,
            visible=False
        )
    )

# Create dropdown buttons: one per neighborhood + "All"
buttons = []

# "All Neighborhoods"
buttons.append(
    dict(
        label="All Neighborhoods",
        method="update",
        args=[
            {"visible": [True] * len(neighborhoods)},
            {"title": "Mean Sale Price per Year – All Neighborhoods"}
        ],
    )
)

# Each neighborhood individually
for i, neigh in enumerate(neighborhoods):
    visible = [False] * len(neighborhoods)
    visible[i] = True
    buttons.append(
        dict(
            label=neigh,
            method="update",
            args=[
                {"visible": visible},
                {"title": f"Mean Sale Price per Year – {neigh}"}
            ],
        )
    )

# Default view: show all
for tr in fig.data:
    tr.visible = True

# ===== 4) Layout + show =====
fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        buttons=buttons,
        x=1.15,
        y=1,
        showactive=True
    )],
    title="Mean Sale Price per Year – All Neighborhoods",
    xaxis_title="Year",
    yaxis_title="Mean Sale Price ($)",
    yaxis_type="log"  # recommended for property prices
)

fig.show()


#### **Sales per Year per Neighbourhood**

#### **Year Built and Sale Price Relationship**

#### **Total Units and Sale Price Relationship**

#### **Price Per Unit**

#### **Sale Price Breakdown per Neighbourhood**